# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [149]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import string
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from joblib import dump, load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
# load data from database
# engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('categorized_messages', 'sqlite:///disasterResponse.db')
X = df.message.values
cat_columns = df.columns[np.logical_not(df.columns.isin(['id', 'message', 'original', 'genre']))]
Y = df[cat_columns].values

### 2. Write a tokenization function to process your text data

In [78]:
# define regex patterns which we'll substitute in our tokenizer
regex = re.compile('[%s]' % re.escape(string.punctuation))

def tokenize(text):
    text = regex.sub(' ', text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok.lower().strip())
        clean_tokens.append(clean_tok)
    # words = [w for w in words if w not in stopwords.words("english")]
    clean_tokens = [tok for tok in clean_tokens if tok not in stopwords.words("english")]
    return clean_tokens

In [79]:
# test tokenizer
test_msg = df['message'][3]
print(test_msg)
tokenize(test_msg)

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.


['un',
 'report',
 'leogane',
 '80',
 '90',
 'destroyed',
 'hospital',
 'st',
 'croix',
 'functioning',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [80]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [83]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [85]:
# print(classification_report(y_true, y_pred, target_names=target_names))
Y_pred = pipeline.predict(X_test)

In [142]:
for cat, ci in zip(cat_columns[1:], range(Y_test.shape[1])[1:]):
    y_test = Y_test[:, ci]
    y_pred = Y_pred[:, ci]
    print("classification report for: {}\n".format(cat))
    print(classification_report(y_test, y_pred))
    


classification report for: request

             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5419
          1       0.82      0.45      0.59      1135

avg / total       0.88      0.89      0.88      6554

classification report for: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

classification report for: aid_related

             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3813
          1       0.74      0.61      0.67      2741

avg / total       0.75      0.75      0.74      6554

classification report for: medical_help

             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6037
          1       0.54      0.13      0.21       517

avg / total       0.90      0.92      0.90 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [144]:
# 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
# 'features__text_pipeline__tfidf__use_idf': (True, False)
parameters = {'vect__max_df': (0.5, 0.75, 1.0), 
              'tfidf__use_idf': (True, False)
             }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [145]:
Y_pred_cv = cv.predict(X_test)

In [146]:
for cat, ci in zip(cat_columns[1:], range(Y_test.shape[1])[1:]):
    y_test = Y_test[:, ci]
    y_pred_cv = Y_pred_cv[:, ci]
    print("classification report for: {}\n".format(cat))
    print(classification_report(y_test, y_pred_cv))

classification report for: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5419
          1       0.82      0.45      0.58      1135

avg / total       0.88      0.89      0.87      6554

classification report for: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

classification report for: aid_related

             precision    recall  f1-score   support

          0       0.74      0.85      0.79      3813
          1       0.74      0.59      0.66      2741

avg / total       0.74      0.74      0.74      6554

classification report for: medical_help

             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6037
          1       0.68      0.13      0.22       517

avg / total       0.91      0.93      0.90 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [148]:
# accuracy
for cat, ci in zip(cat_columns[1:], range(Y_test.shape[1])[1:]):
    y_test = Y_test[:, ci]
    y_pred_cv = Y_pred_cv[:, ci]
    acc = np.mean(y_test == y_pred_cv)
    print("accuracy for {}: {}".format(cat, acc))

accuracy for request: 0.8872444308819042
accuracy for offer: 0.9961855355508087
accuracy for aid_related: 0.7409215746109247
accuracy for medical_help: 0.9266097039975587
accuracy for medical_products: 0.9528532194079952
accuracy for search_and_rescue: 0.9717729630759842
accuracy for security: 0.9830637778455905
accuracy for military: 0.9717729630759842
accuracy for child_alone: 1.0
accuracy for water: 0.9514800122062862
accuracy for food: 0.9240158681721087
accuracy for shelter: 0.9331705828501679
accuracy for clothing: 0.9864205065608789
accuracy for money: 0.9765028989929814
accuracy for missing_people: 0.9893194995422643
accuracy for refugees: 0.9687213915166311
accuracy for death: 0.9594140982606042
accuracy for other_aid: 0.866188587122368
accuracy for infrastructure_related: 0.9353066829417149
accuracy for transport: 0.958193469636863
accuracy for buildings: 0.9560573695453158
accuracy for electricity: 0.9809276777540433
accuracy for tools: 0.9954226426609704
accuracy for hospit

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [150]:
dump(cv, 'categorize_message.joblib') 

['categorize_message.joblib']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.